# Language technology final project
##### Ilse Kerkhove, Marieke Schelhaas & Nikki van Gurp





In [2]:
!python3 -m spacy download nl_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import spacy

nlp = spacy.load("nl_core_news_lg")

In [4]:
import requests
import re
import time

In [20]:
extra_words = {
    'wonen' : 'woonplaats',
    'werken' : 'beroep',
    'lid' : 'onderdeel van',
    'leden' : 'onderdeel van',
    'bestaan' : 'onderdelen',
    'lang' : 'hoogte',
    'zwaar' : 'gewicht',
    'studeren' : 'opleiding'
}

In [41]:
# vinden van de mogelijke entity, de wiki id
def get_entity(sentence):
    url = 'https://www.wikidata.org/w/api.php'
    headers = { 'User-Agent': 'nikkivgurp'}
    params = {'action':'wbsearchentities',
          'language':'nl',
          'uselang':'nl',
          'format':'json'}
    if len(sentence.ents) != 0:
        for ent in sentence.ents:
            params['search'] = ent.text
        try:
          time.sleep(1) # To stop wiki_data from seeing me as a bot
          result = requests.get(url,params).json()
          return result['search'][0]['id']
        except (KeyError, IndexError):
            return None
    else:
        return None

# vinden van de mogelijke properties met Spacy
def find_candidates(sentence):
    candidates = set()
    for token in sentence:
        # using dependency relations
        if token.dep_ in ("attr", "dobj", "popj", "nsubj", "ROOT"):
            candidates.add(token.lemma_)
        elif token.pos_ == "VERB" and token.lemma_ != "zijn":
            candidates.add(token.lemma_)
    for chunk in sentence.noun_chunks:
        candidates.add(chunk.root.text)
        candidates.add(chunk.root.head.text)
        candidates.add(chunk.lemma_)
        candidates.add(chunk.text)
    extra_candidates = {extra_words[word] for word in candidates if word in extra_words}
    candidates.update(extra_candidates)
    return candidates

def get_property(candidates, entity_type):
    url = 'https://www.wikidata.org/w/api.php'
    params = {'action':'wbsearchentities',
          'language':'nl',
          'uselang':'nl',
          'format':'json',
          'type': entity_type}
    if candidates:
        properties = []
        for candidate in candidates:
            params['search'] = candidate
            try:
              time.sleep(1) # To stop wiki_data from seeing me as a bot
              result = requests.get(url,params).json()
              properties.append(result['search'][0]['id'])
            except:
                continue
        return properties
    else:
        return None


# wikidata vinden met Sparql
def get_wikidata(query):
    url = 'https://query.wikidata.org/sparql'
    headers = {'Accept': 'application/json', 'User-Agent': 'nikkivgurp'}
    time.sleep(1) # To stop wiki_data from seeing me as a bot
    results = requests.get(url, params={'query': query, 'format': 'json'}, headers=headers)
    results.raise_for_status()
    return results.json()

# get all possible properties when given an entity
def get_all_p(entity):
  query = 'SELECT ?prop WHERE { wd:' + entity + ' ?prop ?val . FILTER STRSTARTS(STR(?prop), "http://www.wikidata.org/prop/direct/P")}'
  wikidata = get_wikidata(query)
  possible_p_values = set()
  pattern = r'P\d+'
  for item in wikidata['results']['bindings']:
        p_value = re.search(pattern, item['prop']['value'])
        if p_value:
          possible_p_values.add(p_value.group())
  return possible_p_values


# question answer functie
def QA(question):
    sentence = nlp(question)
    properties = get_property(find_candidates(sentence), 'property')
    entity = get_entity(sentence)
    niksgevonden = False
    yes_no_question = False

    # checken of het een ja nee vraag is (vraag die start met werkwoord)
    if sentence[0].pos_ == 'VERB' or sentence[0].pos_ =='AUX':
      yes_no_question = True
      extra_properties = get_property(find_candidates(sentence), 'item')
      extra_properties.extend(properties)

    answers = []
    # Als het een ja of nee-vraag is
    if yes_no_question:
      if extra_properties and entity:
          question_prop = get_all_p(entity)
          if 'P735' in question_prop and 'naam' not in question:
            question_prop.remove('P735')
          for prop in extra_properties:
            for prop_q in question_prop:
              try:
                if len(answers) == 0:
                  yes_no_query = 'ASK WHERE { wd:' + entity + ' wdt:' + prop_q + ' wd:'  + prop + ' . }'
                  response = get_wikidata(yes_no_query)
                  if response['boolean'] == True:
                    answers.append('Ja')
                    break
              except:
                continue
      if len(answers) == 0:
        answers.append('Nee')

    # Overige vragen, waarin entity en property is ontdekt
    elif properties and entity:
        for prop in properties:
            query = '''SELECT ?answer ?answerLabel WHERE {
                       wd:''' + entity + ''' wdt:''' + prop + ''' ?answer .
                       SERVICE wikibase:label {bd:serviceParam wikibase:language "nl" .
                       }
                      }'''
            wikidata = get_wikidata(query)
            bindings = wikidata.get('results', {}).get('bindings', [])
            for result in bindings:
                if 'answerLabel' in result:
                    answers.append(result['answerLabel']['value'])

        # If simple query does not work, try other question types
        if len(answers) == 0:
          if question.split()[0] == "Hoeveel":

            hoeveel_question = True
            # print(sentence[0], "--- ", find_candidates(sentence))
            # print(properties)
          else:
            niksgevonden = True
        else:
          date_pattern = r"^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$"
          if re.match(date_pattern, answers[-1]):
            if 'jaar' in question.lower().split(' '):
              print(question, '\t', answers[-1][:4])
            elif 'datum' in question.lower() or 'dag' in question.lower() or 'wanneer' in question.lower():
              print(question, '\t', answers[-1][8:10], '-', answers[-1][5:7], '-', answers[-1][:4])
            elif 'tijd' in question.lower() or 'hoe laat' in question.lower():
              print(question, '\t', answers[-1][-9:-1])
            else:
              print(question, '\t', answers[-1])
          else:
            print(question, '\t', answers[-1]) # uiteindelijke antwoord
    else:
      niksgevonden = True

    if yes_no_question:
      print(question, '\t', answers[-1])
    elif niksgevonden:
       print(question, '\t', 'Niks gevonden')





# Debugging Testsets
## General Question-set
q1 = 'Welke talen spreekt Arjen Lubach?'
q2 = 'Wanneer is Jan Smit geboren?'
q3 = 'Wanneer is Rembrandt van Rijn overleden?'
q4 = 'Hoe heet het kind van Michiel de Ruyter?'
question_list1 = [q1, q2, q3, q4]

## Hoe Question-set
q5 = 'Hoeveel volgers heeft Jan Smit?'
q6 = 'Hoe oud is Mark Rutte?'
q7 = 'Hoeveel onderscheidingen heeft Froukje?'
q14 = 'Hoeveel kinderen heeft Dick Schoof?'
question_list2 = [q14, q5, q7, q6]

## Lijst Question-set
q8 = 'Welke talen spreek Arjen Lubach'
q9 = '?'
q10 = '?'
question_list3 = [q8, q9, q10]

## Ja/Nee Question-set
q11 = '?'
q12 = '?'
q13 = '?'
question_list4 = [q11, q12, q13]

question = input("Stel een vraag over een BN'er\n")
# hij doet er wel een paar seconde over

# for question in question_list2:
QA(question)

Stel een vraag over een BN'er
Is Roxy Dekker zangeres?
Is Roxy Dekker zangeres? 	 Ja
